In [132]:
from __future__ import print_function
import datetime
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from dateutil.parser import parse



In [87]:
from apiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow

In [88]:
scopes = ['https://www.googleapis.com/auth/calendar']

In [89]:
flow = InstalledAppFlow.from_client_secrets_file("client_secret2.json", scopes=scopes)
credentials = flow.run_console()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=167580570288-7sk7lav2io2g18ucah0oe11hkbtbrulm.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&state=1euY0XgXu15auHg8ukdi4CGp6hDLcR&prompt=consent&access_type=offline
Enter the authorization code: 4/zwFWrn-MDZslXq5XJCOyoV4GUj8Ju5oOkZY0fYVADTNFU0wp3DRLTzU


In [90]:
pickle.dump(credentials, open("token.pkl", "wb")) 
credentials = pickle.load(open("token.pkl", "rb"))

In [91]:
service = build('calendar', 'v3', 
                credentials=credentials)

In [98]:
now = datetime.datetime.utcnow().isoformat() + 'Z' # 'Z' indicates UTC time
print('Getting the upcoming 10 events')
events_result = service.events().list(calendarId='primary', timeMin=now,
                                    maxResults=10, singleEvents=True,
                                    orderBy='startTime').execute()
events = events_result.get('items', [])

if not events:
    print('No upcoming events found.')
for event in events:
    start = event['start'].get('dateTime', event['start'].get('date'))
    print(event['summary'],event['id'])


Getting the upcoming 10 events
Wisdom Teeth removal 5n9jif0jr2avv04348u2ehn26j
IIT C Webinar 7.30PM 60h9kg0pv3oo6tvagec51jv484
Jagan Mohan birthday coo30c9g6cp68b9jchi62b9k70r64bb26gojebb5cooj2p1i6lj6aohkc8_20200621T180000Z
UIC Intent of Enroll 2kvjs3mnl63sr6b92et8qklo8r
Sumedha Khurana birthday  68ojiohl60p38b9n61ij0b9k6ssm4b9oc5i3ab9m65hm8d1jcdj68d1mcg_20200705T175900Z
Preeteshwar singh birthday  64rj2cj669gjabb364pm8b9k6lh3gbb265j32bb16ph3iohncgo3aor664_20200722
Ritesh Tripathy birthday  c5im8p336di36b9j69imab9k75i38b9o68qjeb9p6lij4pj2c5gm4cj16c_20200831
Padmini Sharma birthday 6oom4cppcgqjab9mccsm8b9kchi66bb270s62b9nc9gj6d9gc8o3adpock_20200912T163000Z
Amish singh birthday  74p32o9p74q64bb1chhmcb9k6orm2b9pc4o66b9l6cs3gdj26opmaphn6o_20200916
Sushant shekhar birthday  70pmap9j6oojib9p6kr66b9kcgr62b9pcphjgb9jcks38c9icor68cj46s_20200920


In [46]:
import numpy as np
import pandas as pd

In [47]:
df= pd.read_csv('Results.csv',sep=',')

In [48]:
df.head()

,Security Code,Security Name,Company name,Result Date
0,532100,INDOCITY,INDO-CITY INFOTECH LTD.-$,16 May 2020
1,504112,NELCO,NELCO LTD.,16 May 2020
2,521064,TRIDENT,TRIDENT LTD.,16 May 2020
3,530131,UDAICEMENT,UDAIPUR CEMENT WORKS LTD.,16 May 2020
4,532633,ALLSEC,ALLSEC TECHNOLOGIES LTD.,18 May 2020


In [102]:
for i in df.head().index :
    event = {
  'summary': df['Security Name'][i],
  'description': 'BSE Results',
  'start': {
    'dateTime': parse(df['Result Date'][i]).isoformat()+"Z",
    
  },
  'end': {
    'dateTime': parse(df['Result Date'][i]).isoformat()+"Z",
    'timeZone': 'America/Los_Angeles',
  },
}
    event = service.events().insert(calendarId='primary', body=event).execute()
    print (df['Security Name'][i], parse(df['Result Date'][i]).isoformat()+"Z")

INDOCITY 2020-05-16T00:00:00Z
NELCO 2020-05-16T00:00:00Z
TRIDENT 2020-05-16T00:00:00Z
UDAICEMENT 2020-05-16T00:00:00Z
ALLSEC 2020-05-18T00:00:00Z


In [94]:
print(event['id'])

74p32o9p74q64bb1chhmcb9k6orm2b9pc4o66b9l6cs3gdj26opmaphn6o_20200916


In [97]:
service.events().delete(calendarId='primary', eventId="7v6trqoeor0v6adtilap8jtupp").execute()

''

In [130]:
EVENT = {
    'start':  {'dateTime': parse(df['Result Date'][4]).isoformat()+"Z"},
    'end':    {'dateTime': parse(df['Result Date'][4]).isoformat()+"Z"},
    
}

In [118]:
page_token = None
while True:
    events = service.events().list(calendarId='primary', pageToken=page_token).execute()
    for event in events['items']:
        if(event['summary']==df['Security Name'][3]):
            print(event['id'])
            service.events().update(calendarId='primary',eventId=event['id'], body=EVENT).execute()
    page_token = events.get('nextPageToken')
    if not page_token:
        break

p7a1i4lq3mdtiep20uo1fu1s3c


KeyError: 'summary'

In [131]:
eventsList=service.events().list(calendarId='primary', pageToken=page_token).execute()

for event in eventsList['items']:
    if(event['summary']==df['Security Name'][0]):
        EVENT = {
            'summary': df['Security Name'][0],
    'start':  {'dateTime': parse(df['Result Date'][4]).isoformat()+"Z"},
    'end':    {'dateTime': parse(df['Result Date'][4]).isoformat()+"Z"},
    
}
        service.events().update(calendarId='primary',eventId=event['id'], body=EVENT).execute()
